<a href="https://colab.research.google.com/github/miaojingang/private_ratio/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook has the simulation used in paper

**to be added.**

In [ ]:
#@title functions {form-width:"20%"}
import logging
import numpy as np
import pandas as pd
import functools
import warnings
warnings.filterwarnings('ignore')


def from_moments(m_s, m_y, v_s, v_y, v_ys, log_scale=False):
    """Gets mean and variance from moments of the means."""
    if log_scale:
        return np.log(m_s / m_y), (
            1. / m_s ** 2 * v_ys
            -2 * v_ys / m_y / m_s
            + v_y / m_y ** 2
        )
    else:
        return m_s / m_y, (
            1. / m_y ** 2 * v_s
            - 2 * m_s / m_y **3 * v_ys
            + m_s ** 2 / m_y** 4 * v_y
        )


def moments_from_sums(s, s2, y, y2, ys, w, w2):
    """Gets moments of the means from sums."""
    effective_n = w ** 2 / w2
    return (
        s / w, y / w,  # m_s, m_y
        (s2 / w - (s / w) ** 2) / effective_n,  # v_s
        (y2 / w - (y / w) ** 2) / effective_n,  # v_y
        (ys / w - y * s / w ** 2) / effective_n  # v_ys
    )

def from_sums(s, s2, y, y2, ys, w, w2, log_scale=False):
    """Gets mean and variance from sums."""
    moments = moments_from_sums(s, s2, y, y2, ys, w, w2)
    return from_moments(*moments, log_scale=log_scale)

def get_ci(m, v):
    half = 1.96 * np.sqrt(v)
    return m - half, m + half

def ci_length(ci):
    return ci[1] - ci[0]

def is_covered(ci, truth):
    return ci[0] < truth < ci[1]

def get_noise_sd(Delta, epsilon, delta):
    return Delta * np.sqrt(2 * np.log(1.25 / delta)) / epsilon

def one_run_dp_sums(
    seed=0, n=1_000, u_w=1, B=200,
    epsilon=2, delta=1e-6, true=1.1, log_scale=False):
    # seed=0; n=1_000; u_w=1; B=200; epsilon=2; delta=1e-6; true=1.1; log_scale=False
    np.random.seed(seed)
    s = np.random.beta(2, 2, size=n)
    y = np.random.binomial(n=1, p=s/true, size=n)
    l_w = 1 / u_w
    w = np.random.exponential(1., size=n).clip(l_w, u_w)

    if log_scale:
        true = np.log(true)
        maybe_log = np.log
    else:
        maybe_log = lambda x: x

    # public version
    sum_s = s @ w
    sum_s2 = (s ** 2) @ w
    sum_y = sum_y2 = y @ w
    sum_ys = (y * s) @ w
    sum_w = w.sum()
    sum_w2 = (w ** 2).sum()
    m_p, v_p = from_sums(
        sum_s, sum_s2, sum_y, sum_y2,
        sum_ys, sum_w, sum_w2, log_scale=log_scale
    )
    ci_p = get_ci(m_p, v_p)
    length_p = ci_length(ci_p)
    covered_p = is_covered(ci_p, true)

    # DP: add normal noise
    if u_w == 1.0:
        get_sd = functools.partial(
            get_noise_sd, epsilon=epsilon / 5, delta = delta / 5)
    else:
        get_sd = functools.partial(
            get_noise_sd, epsilon=epsilon / 6, delta = delta / 6)
    
    sum_s += np.random.normal(0, get_sd(u_w))
    sum_s2 += np.random.normal(0, get_sd(u_w))
    sum_y += np.random.normal(0, get_sd(u_w))
    sum_y2 = sum_y
    sum_ys += np.random.normal(0, get_sd(u_w))
    sum_w += np.random.normal(0, get_sd(u_w))
    if u_w == 1.0:
        sum_w2 = sum_w
    else:
        sum_w2 += np.random.normal(0, get_sd(u_w ** 2))
    # no correction
    m, v_nc = from_sums(
        sum_s, sum_s2, sum_y, sum_y2,
        sum_ys, sum_w, sum_w2, log_scale=log_scale
    )
    bias = m - true
    effective_n = w.sum() ** 2 / (w ** 2).sum()
    ci_nc = get_ci(m, v_nc)
    length_nc = ci_length(ci_nc)
    covered_nc = is_covered(ci_nc, true)

    # monte carlo correction
    v_extra = ((maybe_log(
        (sum_s + np.random.normal(0, get_sd(u_w), size=B)) /
        (sum_y + np.random.normal(0, get_sd(u_w), size=B))
        ) - m
    ) ** 2).mean()
    
    ci_m = get_ci(m, v_nc + v_extra)
    length_m = ci_length(ci_m)
    covered_m = is_covered(ci_m, true)

    # analytical correction
    m_s, m_y, v_s, v_y, v_sy = moments_from_sums(sum_s, sum_s2, sum_y, sum_y2,
        sum_ys, sum_w, sum_w2)
    _, v_a = from_moments(
        m_s * sum_w, m_y * sum_w,
        v_s * sum_w ** 2 + get_sd(u_w) ** 2,
        v_y * sum_w ** 2 + get_sd(u_w) ** 2,
        v_sy * sum_w ** 2, log_scale=log_scale
    )
    ci_a = get_ci(m, v_a)
    length_a = ci_length(ci_a)
    covered_a = is_covered(ci_a, true)

    return (
        n, epsilon, delta,
        u_w, effective_n,
        length_p, covered_p,
        length_nc, covered_nc,
        length_m, covered_m,
        length_a, covered_a,
        bias
    )

def sim(
    show_styled_result=True,
    show_latex_result=True,
    **kwargs):
    res = (
        pd.DataFrame.from_records(
            (one_run_dp_sums(seed, n=n, u_w=u_w, epsilon=epsilon, **kwargs)
            for seed in range(1_000)
            for u_w in [1, 5]
            for n in [5000, 10_000]
            for epsilon in [0.5, 1., 4., 100.]
            ),
            columns = [
                "n", "epsilon", "delta", "u_w", "effective_n",
                "length_p", "coverage_p",
                "length_nc", "coverage_nc",
                "length_m", "coverage_m",
                "length_a", "coverage_a",
                "bias"]
            )
        .groupby(["n", "u_w", "epsilon", "delta", ], as_index=False).mean()
    )
    if show_styled_result:
        display(
            res.style.hide_index()
            .format({
                "n": "{:,}",
                "epsilon": "{:.1f}", "effective_n": "{:,.0f}",
                "length_p": "{:.3f}", "coverage_p": "{:.3f}",
                "length_nc": "{:.3f}", "coverage_nc": "{:.3f}",
                "length_m": "{:.3f}", "coverage_m": "{:.3f}",
                "length_a": "{:.3f}", "coverage_a": "{:.3f}",
                "bias": "{:.3f}"
            })
            .bar(subset=["coverage_p", "coverage_nc", "coverage_m", "coverage_a"],
                 color="lightgreen", vmin=0)
            .set_table_attributes("class='dataframe'")
        )
    if show_latex_result:
        cols = [
            "u_w", "effective_n",
            "n", "epsilon",
             "length_p", "coverage_p",
            "length_nc", "coverage_nc",
            "length_m", "coverage_m",
            "length_a", "coverage_a"
        ]
        print(res[cols].round(3).to_latex())

In [ ]:
#@title Simulation shown in paper {form-width:"20%"}
sim(delta=1e-6, B=200)

n,u_w,epsilon,delta,effective_n,length_p,coverage_p,length_nc,coverage_nc,length_m,coverage_m,length_a,coverage_a,bias
"5,000",1,0.5,0.000001,"5,000",0.061,0.947,0.061,0.575,0.156,0.949,0.156,0.949,0.001
"5,000",1,1.0,0.000001,"5,000",0.061,0.947,0.061,0.784,0.094,0.947,0.094,0.947,0.000
"5,000",1,4.0,0.000001,"5,000",0.061,0.947,0.061,0.936,0.064,0.948,0.064,0.948,0.000
"5,000",1,100.0,0.000001,"5,000",0.061,0.947,0.061,0.948,0.061,0.948,0.061,0.948,0.000
"5,000",5,0.5,0.000001,"2,662",0.084,0.947,0.086,0.142,0.989,0.949,0.903,0.946,0.023
"5,000",5,1.0,0.000001,"2,662",0.084,0.947,0.083,0.312,0.447,0.947,0.441,0.947,0.006
"5,000",5,4.0,0.000001,"2,662",0.084,0.947,0.084,0.776,0.136,0.946,0.136,0.949,0.000
"5,000",5,100.0,0.000001,"2,662",0.084,0.947,0.084,0.942,0.084,0.943,0.084,0.943,-0.000
"10,000",1,0.5,0.000001,"10,000",0.043,0.955,0.043,0.665,0.083,0.934,0.084,0.935,-0.001
"10,000",1,1.0,0.000001,"10,000",0.043,0.955,0.043,0.874,0.056,0.951,0.056,0.947,-0.001


\begin{tabular}{lrrrrrrrrrrrr}
\toprule
{} &  u\_w &  effective\_n &      n &  epsilon &  length\_p &  coverage\_p &  length\_nc &  coverage\_nc &  length\_m &  coverage\_m &  length\_a &  coverage\_a \\
\midrule
0  &    1 &     5000.000 &   5000 &      0.5 &     0.061 &       0.947 &      0.061 &        0.575 &     0.156 &       0.949 &     0.156 &       0.949 \\
1  &    1 &     5000.000 &   5000 &      1.0 &     0.061 &       0.947 &      0.061 &        0.784 &     0.094 &       0.947 &     0.094 &       0.947 \\
2  &    1 &     5000.000 &   5000 &      4.0 &     0.061 &       0.947 &      0.061 &        0.936 &     0.064 &       0.948 &     0.064 &       0.948 \\
3  &    1 &     5000.000 &   5000 &    100.0 &     0.061 &       0.947 &      0.061 &        0.948 &     0.061 &       0.948 &     0.061 &       0.948 \\
4  &    5 &     2662.409 &   5000 &      0.5 &     0.084 &       0.947 &      0.086 &        0.142 &     0.989 &       0.949 &     0.903 &       0.946 \\
5  &    5 &     2

In [ ]:
#@title log_scale results {form-width: "20%"}
sim(log_scale=True, show_latex_result=False, delta=1e-6, B=200)

n,u_w,epsilon,delta,effective_n,length_p,coverage_p,length_nc,coverage_nc,length_m,coverage_m,length_a,coverage_a,bias
"5,000",1,0.5,0.000001,"5,000",0.055,0.946,0.055,0.571,0.141,0.948,0.111,0.884,0.000
"5,000",1,1.0,0.000001,"5,000",0.055,0.946,0.055,0.789,0.085,0.944,0.073,0.898,0.000
"5,000",1,4.0,0.000001,"5,000",0.055,0.946,0.055,0.932,0.057,0.945,0.056,0.942,0.000
"5,000",1,100.0,0.000001,"5,000",0.055,0.946,0.055,0.945,0.055,0.945,0.055,0.945,0.000
"5,000",5,0.5,0.000001,"2,662",0.075,0.943,0.074,0.148,0.822,0.967,0.592,0.870,0.002
"5,000",5,1.0,0.000001,"2,662",0.075,0.943,0.075,0.316,0.399,0.955,0.299,0.863,0.000
"5,000",5,4.0,0.000001,"2,662",0.075,0.943,0.075,0.775,0.123,0.949,0.104,0.902,-0.000
"5,000",5,100.0,0.000001,"2,662",0.075,0.943,0.075,0.940,0.075,0.940,0.075,0.940,-0.000
"10,000",1,0.5,0.000001,"10,000",0.039,0.953,0.039,0.664,0.076,0.933,0.062,0.877,-0.001
"10,000",1,1.0,0.000001,"10,000",0.039,0.953,0.039,0.871,0.051,0.950,0.046,0.919,-0.001
